<a href="https://colab.research.google.com/github/sochachai/RankNet/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installation of Dependencies

In [2]:
!pip install tensorflow==2.12.0
!pip install tensorflow_hub==0.12.0
!pip install tensorflow_text
!pip install transformers
!pip install sentence_transformers
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 622.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Succ

Check versions

In [3]:
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
import tensorflow_hub as hub
print("Hub version:", hub.__version__)


Tensorflow version: 2.16.1


AttributeError: module 'tensorflow._api.v2.compat.v1' has no attribute 'estimator'

In [5]:
!pip uninstall tensorflow
!pip install tensorflow==2.12.0

  Using cached tensorflow-2.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (585.9 MB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.16.1 requires tensorflow<2.17,>=2.16.1; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.12.0 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.12.0 which is incompatible.


Check versions again (after pressing RESTART SESSION in the previous block)





In [1]:
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
import tensorflow_hub as hub
print("Hub version:", hub.__version__)


Tensorflow version: 2.12.0
Hub version: 0.12.0


In [4]:
!pip install tensorflow_text==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow_text
    Found existing installation: tensorflow-text 2.16.1
    Uninstalling tensorflow-text-2.16.1:
      Successfully uninstalled tensorflow-text-2.16.1


Load dependencies

In [5]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
import tensorflow_text
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
nltk.download("averaged_perceptron_tagger")
from transformers import DistilBertModel, DistilBertTokenizer
import torch


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Go to selected directory

In [6]:
%cd /content/drive/MyDrive/Recommendation_System

/content/drive/MyDrive/Recommendation_System


Load Data

In [53]:
df = pd.read_csv('flipkart_com-ecommerce_sample.csv')
print(df.head(2))

                            uniq_id            crawl_timestamp  \
0  c2d766ca982eca8304150849735ffef9  2016-03-25 22:59:23 +0000   
1  7f7036a6d550aaa89d34c77bd39a5e48  2016-03-25 22:59:23 +0000   

                                         product_url  \
0  http://www.flipkart.com/alisha-solid-women-s-c...   
1  http://www.flipkart.com/fabhomedecor-fabric-do...   

                          product_name  \
0  Alisha Solid Women's Cycling Shorts   
1  FabHomeDecor Fabric Double Sofa Bed   

                               product_category_tree               pid  \
0  ["Clothing >> Women's Clothing >> Lingerie, Sl...  SRTEH2FF9KEDEFGF   
1  ["Furniture >> Living Room Furniture >> Sofa B...  SBEEH3QGU7MFYJFY   

   retail_price  discounted_price  \
0         999.0             379.0   
1       32157.0           22646.0   

                                               image  is_FK_Advantage_product  \
0  ["http://img5a.flixcart.com/image/short/u/4/a/...                    False   
1  ["htt

In [54]:
print(df.columns)

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')


In [55]:
df = df[['pid','product_name','brand','description','discounted_price',\
         'product_specifications']]
product_name_df = df['product_name']

In [56]:
df.head()

,pid,product_name,brand,description,discounted_price,product_specifications
0,SRTEH2FF9KEDEFGF,Alisha Solid Women's Cycling Shorts,Alisha,Key Features of Alisha Solid Women's Cycling S...,379.0,"{""product_specification""=>[{""key""=>""Number of ..."
1,SBEEH3QGU7MFYJFY,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,22646.0,"{""product_specification""=>[{""key""=>""Installati..."
2,SHOEH4GRSUBJGZXE,AW Bellies,AW,Key Features of AW Bellies Sandals Wedges Heel...,499.0,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,SRTEH2F6HUZMQ6SJ,Alisha Solid Women's Cycling Shorts,Alisha,Key Features of Alisha Solid Women's Cycling S...,267.0,"{""product_specification""=>[{""key""=>""Number of ..."
4,PSOEH3ZYDMSYARJ5,Sicons All Purpose Arnica Dog Shampoo,Sicons,Specifications of Sicons All Purpose Arnica Do...,210.0,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pid                     20000 non-null  object 
 1   product_name            20000 non-null  object 
 2   brand                   14136 non-null  object 
 3   description             19998 non-null  object 
 4   discounted_price        19922 non-null  float64
 5   product_specifications  19986 non-null  object 
dtypes: float64(1), object(5)
memory usage: 937.6+ KB


Drop duplications

In [58]:
print(df.duplicated().value_counts())
df.drop_duplicates(inplace=True)
print(f"Shape after the deletion of duplicates is {df.shape}")

False    20000
Name: count, dtype: int64
Shape after the deletion of duplicates is (20000, 6)


Switch datatype to strings and in lower cases

In [59]:
df = df.astype(str)
df = df.applymap(lambda x : x.lower())

Raw text data in a format with key-value pairs

In [60]:
df['product_specifications'][0]

'{"product_specification"=>[{"key"=>"number of contents in sales package", "value"=>"pack of 3"}, {"key"=>"fabric", "value"=>"cotton lycra"}, {"key"=>"type", "value"=>"cycling shorts"}, {"key"=>"pattern", "value"=>"solid"}, {"key"=>"ideal for", "value"=>"women\'s"}, {"value"=>"gentle machine wash in lukewarm water, do not bleach"}, {"key"=>"style code", "value"=>"altht_3p_21"}, {"value"=>"3 shorts"}]}'

In [61]:
def extraction_specifications(specifications):
    #extract key and value from product_specification column
    pairs = re.findall(r'"key"=>"(.*?)", "value"=>"(.*?)"',specifications)
    pairs_formatted = [f"{key} : {value}" for key, value in pairs]
    return ' '.join(pairs_formatted)

df['product_specifications'] = df['product_specifications'].apply(extraction_specifications)
print(df['product_specifications'][0])


number of contents in sales package : pack of 3 fabric : cotton lycra type : cycling shorts pattern : solid ideal for : women's style code : altht_3p_21


Color in brands are irrelevant

In [62]:
def remove_colors(brand):
    colors = ['amber','aquamarine','aqua','beige','black','blue','bronze',\
              'brown','burgundy','caramel','chartreuse','charcoal','coral',\
              'crimson','cyan','eggplant','emerald','forest green','fuchsia',\
              'gold','golden','gray','green','indigo','ivory','jade','khaki',\
              'lavender','lemon','lime','lilac','magenta','maroon','mint',\
              'mint green','moss green','mustard','navy','neon amber',\
              'neon blue','neon coral','neon crimson','neon fuchsia','neon gold',\
              'neon green', 'neon indigo','neon lime','neon magenta','neon orange',\
              'neon pink','neon platinum','neon purple','neon red','neon silver',\
              'neon teal','neon turquoise','neon violet','olive','olive green',\
              'orange','orchid','pearl','peach','peacock blue','peacock green',\
              'periwinkle','pink','platinum','purple','red','rose','rose gold',\
              'ruby','ruby red','rust','salmon','salmon pink','sapphire',\
              'silver','sky blue','steel blue','tan','tangerine','taupe','teal',\
              'topaz','turqoise','violet','white','yellow','baby pink','baby blue']
    pattern = r'\b(?:{})\b'.format('|'.join(colors))
    #print(pattern)
    brand = re.sub(pattern, '', brand, flags=re.IGNORECASE).strip()
    return brand

df['brand'] = df['brand'].apply(remove_colors)

Get combined text

In [63]:
df = df.applymap(lambda x: re.sub('[^A-Za-z0-9]+', ' ', str(x)))
df['combined_text'] = df['product_name'] + ' ' + df['description'] + ' ' + df['product_specifications']

In [64]:
df['combined_text'][0]

'alisha solid women s cycling shorts key features of alisha solid women s cycling shorts cotton lycra navy red navy specifications of alisha solid women s cycling shorts shorts details number of contents in sales package pack of 3 fabric cotton lycra type cycling shorts general details pattern solid ideal for women s fabric care gentle machine wash in lukewarm water do not bleach additional details style code altht 3p 21 in the box 3 shorts number of contents in sales package pack of 3 fabric cotton lycra type cycling shorts pattern solid ideal for women s style code altht 3p 21'

Load Transformer

In [65]:
from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('bert-base-nli-mean-tokens')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Truncate the data

In [68]:
df = df.iloc[0:50,:]

In [69]:
# Get embeddings
sentence_embeddings = bert.encode(df['combined_text'].tolist())
# Define path to save embeddings
embeddings_path = "/content/drive/MyDrive/Recommendation_System/embeddings3.npy"

In [70]:
def extract_brand(query):
    query = str(query).lower()
    brand_names = df['brand'].str.lower()
    for brand in brand_names:
        if brand in query:
            return brand
    return None


In [71]:
np.save(embeddings_path, sentence_embeddings)

Use model to establish a recommendation system

In [76]:
sentence_embeddings = np.load(embeddings_path)
# prevent truncation from end of product name
#pd.set_option('display.max_colwidth',-1)
def brand_inference(user_input,sentence_embeddings):
    print("Recommendations for \"", user_input, "\" are:", sep='')
    user_input = re.sub('[^A-Za-z0-9]+', ' ', user_input)
    start = time.time()
    query_brand = extract_brand(user_input)
    user_input_embedding = bert.encode([user_input])
    similarities = cosine_similarity(user_input_embedding,sentence_embeddings)
    if query_brand is not None:
        df['brand_match_weight'] = df['brand'].apply(lambda x:1.5 if x==query_brand else 1.0)
        df['brand_match_weight'] = df['brand_match_weight'] * df['product_name'].apply(lambda x:1.5\
                                                                        if query_brand in x else 1.0)
    else:
        df['brand_match_weight'] = 1.0
    similarities_with_brand = similarities * (np.array(df['brand_match_weight']))
    num_recommendations = 5
    top_indices = similarities_with_brand.argsort()[0][::-1][:num_recommendations]
    df['product_name_origin'] = product_name_df[0:len(df)]
    recommended_products = df.iloc[top_indices]['product_name_origin']
    print(recommended_products)
    end = time.time()
    print(f"Time spent is {end-start}.")

In [77]:
input_list = ["bluetooth headphones wired and wireless", "asics running shoes", "T-Shirts", "car mat"]
for i in input_list:
    brand_inference(i, sentence_embeddings)
    print()

Recommendations for "bluetooth headphones wired and wireless" are:
43                    Rapter BNC-179 BNC Wire Connector
47    BuildTrack PIR Wireless Motion Sensor - One Sw...
41    OM SHIVAKRITI Square wall Clock Showpiece  -  ...
10                                       Ladela Bellies
42    Himmlisch ST381 Magnetic Sun Shade For Maruti ...
Name: product_name_origin, dtype: object
Time spent is 0.5978944301605225.

Recommendations for "asics running shoes" are:
40                           Nuride Canvas Shoes
23                            SWAGGA Women Clogs
39                            Oye Boy's Dungaree
20    Sicons Conditioning Conditoner Dog Shampoo
43             Rapter BNC-179 BNC Wire Connector
Name: product_name_origin, dtype: object
Time spent is 0.18039941787719727.

Recommendations for "T-Shirts" are:
40                          Nuride Canvas Shoes
35          KAJCI Embroidered Women's Waistcoat
37    Pick Pocket Embroidered Women's Waistcoat
23                          